# Material Recognition using Transfer Learning on VGG19

The goal of this project is to train a convolutional neural network to classify color photographs of surfaces into one of ten common material categories: fabric, foliage, glass, leather, metal, paper, plastic, stone, water, and wood. Some tasks to consider:

1. Modify some previously published architectures e.g., increase the network depth, reducing their parameters, etc.
2. Try data augmentation to increase the number of training images
3. Try a larger dataset, Materials in Context Database (MINC)

Dataset: Flickr Material Database (FMD)

The notebook here will focus on transfer learning using VGG-19. The overview of this notebook is as follows:

1. Preprocessing

2. Train-Val-Test split

3. Defining base model architecture using VGG-19

4. Transfer Learning on VGG-19 with Adam Optimiser (all frozen layers), hyperparameter tuning it using Keras-Tuner, evaluation on test dataset

5. Transfer Learning on VGG-19 with SGD Optimiser (all frozen layers) and hyperparameter tuning it using Keras-Tuner, evaluation on test dataset

6. Transfer Learning on VGG-19 with Adam Optimiser (with some unfrozen layers), hyperparameter tuning it using Keras-Tuner, evaluation on test dataset

7. Transfer Learning on VGG-19 with SGD Optimiser (with some unfrozen layers) and hyperparameter tuning it using Keras-Tuner, evaluation on test dataset

8. New Train-Val_Test split after addition of images generated using GAN

9. Transfer Learning on VGG-19 with Adam Optimiser with GAN (all frozen layers), hyperparameter tuning it using Keras-Tuner, evaluation on test dataset

10. Transfer Learning on VGG-19 with SGD Optimiser with GAN (all frozen layers) and hyperparameter tuning it using Keras-Tuner, evaluation on test dataset

11. Transfer Learning on VGG-19 with Adam Optimiser with GAN (with some unfrozen layers), hyperparameter tuning it using Keras-Tuner, evaluation on test dataset

12. Transfer Learning on VGG-19 with SGD Optimiser with GAN(with some unfrozen layers) and hyperparameter tuning it using Keras-Tuner, evaluation on test dataset


# Preprocessing

In [4]:
import keras_tuner as kt
import tensorflow as tf
from tensorflow.keras.layers import Dense, Flatten, Dropout
from tensorflow.keras import Sequential

## Train-validation-test split 

In [2]:
materials = ['fabric','foliage','glass','leather','metal','paper','plastic','stone','water','wood']

path_to_folder = '../input/fmdmaterials/image'
def train_val_dataset(validation_split, seed):
    
    train_dataset = tf.keras.utils.image_dataset_from_directory(
    directory=path_to_folder,
    labels='inferred',
    label_mode='categorical',
    class_names=materials,
    # color_mode='grayscale',
    batch_size=32,
    image_size=(224,224),
    shuffle=True,
    seed=seed,
    validation_split=validation_split,
    subset='training')
    
    
    validation_dataset = tf.keras.utils.image_dataset_from_directory(
    directory=path_to_folder,
    labels='inferred',
    label_mode='categorical',
    class_names=materials,
    # color_mode='grayscale',
    batch_size=32,
    image_size=(224,224),
    shuffle=True,
    seed=seed,
    validation_split=validation_split,
    subset='validation'
)
    
    return train_dataset, validation_dataset
    
    
    

In [21]:
test_dataset = tf.keras.utils.image_dataset_from_directory(
    directory='../input/testmaterial/test/data/test',
    labels='inferred',
    label_mode='categorical',
    class_names=materials,
    # color_mode='grayscale',
    batch_size=32,
    image_size=(224,224),
    shuffle=True)

Found 30 files belonging to 10 classes.


In [3]:
train_dataset, validation_dataset= train_val_dataset(0.3, 42)

Found 1000 files belonging to 10 classes.
Using 700 files for training.
Found 1000 files belonging to 10 classes.
Using 300 files for validation.


2022-11-10 21:04:20.396253: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-11-10 21:04:20.574121: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-11-10 21:04:20.575074: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-11-10 21:04:20.578519: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compil

## Data augmentation

In [15]:
data_augmentation = tf.keras.Sequential([
  tf.keras.layers.RandomFlip("vertical"),
  tf.keras.layers.RandomRotation(0.2),
])

## Using pre-trained VGG19 as base model

In [6]:
image_shape = (224,224) + (3,)
base_model = tf.keras.applications.vgg19.VGG19(input_shape=image_shape,
                                               include_top=False,
                                               weights='imagenet')

80150528/80134624 [==============================] - 0s 0us/step


In [7]:
preprocess_input = tf.keras.applications.vgg19.preprocess_input

In [8]:
base_model.summary()

Model: "vgg19"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0     

# Transfer Learning and tune the hyperparameters with frozen layers (Adam optimizer)

In [24]:
base_model.trainable = False

In [12]:
def model_builder(hp):
    
    hp_units = hp.Choice('units', [16, 32, 64, 128, 256, 512, 1024])
    hp_lr = hp.Choice('lr', [1e-2, 1e-3, 1e-4])
    hp_ratio1 = hp.Choice('ratio1', [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8])
    hp_ratio2 = hp.Choice('ratio2', [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8])
    hp_activation = hp.Choice('activation', ['relu', 'elu'])
        
    inputs = tf.keras.Input(shape=(224, 224, 3))
    
    x = data_augmentation(inputs)
    x = preprocess_input(x)
    x = base_model(x, training=False)
    x = Flatten()(x)
    x = Dropout(hp_ratio1)(x)
    x = Dense(units=hp_units, activation=hp_activation)(x)
    x = Dropout(hp_ratio2)(x)
    prediction = Dense(10, activation='softmax')(x)
    model = tf.keras.Model(inputs=inputs, outputs=prediction)
    
    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=hp_lr),
        loss=tf.keras.losses.CategoricalCrossentropy(),
        metrics=['accuracy']
    )

    return model

In [26]:
tuner = kt.Hyperband(
    model_builder,
    objective='val_accuracy',
    max_epochs=10,
    directory="./",
)

In [27]:
stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)

In [28]:
tuner.search(train_dataset, epochs=50,validation_data=validation_dataset, callbacks=[stop_early])

# Get the optimal hyperparameters
best_hps=tuner.get_best_hyperparameters(num_trials=1)[0]

Trial 30 Complete [00h 00m 47s]
val_accuracy: 0.11666666716337204

Best val_accuracy So Far: 0.7599999904632568
Total elapsed time: 00h 10m 46s


In [29]:
print(best_hps.get('units'))
print(best_hps.get('lr'))
print(best_hps.get('ratio1'))
print(best_hps.get('ratio2'))
print(best_hps.get('activation'))

1024
0.001
0.4
0.6
elu


In [31]:
inputs = tf.keras.Input(shape=(224, 224, 3))
x = data_augmentation(inputs)
x = preprocess_input(x)
x = base_model(x, training=False)
x = Flatten()(x)
x = tf.keras.layers.Dropout(0.4)(x)
x = tf.keras.layers.Dense(units=1024, activation='elu')(x)
x = tf.keras.layers.Dropout(0.6)(x)
prediction = Dense(10, activation='softmax')(x)
model = tf.keras.Model(inputs=inputs, outputs=prediction)

model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate= 0.001),
              loss=tf.keras.losses.CategoricalCrossentropy(),
              metrics=['accuracy'])

model_checkpoints = [
    tf.keras.callbacks.ModelCheckpoint(filepath = 'hyperparameter_adam.h5', 
    verbose=1, save_best_only=True, save_weights_only=False, mode='max', monitor='val_accuracy') 
    ]

history = model.fit(train_dataset,epochs=50, validation_data = validation_dataset, callbacks = [model_checkpoints])


Epoch 1/50
22/22 [==============================] - 5s 164ms/step - loss: 44.9413 - accuracy: 0.3829 - val_loss: 18.8146 - val_accuracy: 0.6633

Epoch 00001: val_accuracy improved from -inf to 0.66333, saving model to hyperparameter_adam.h5
Epoch 2/50
22/22 [==============================] - 4s 152ms/step - loss: 16.3732 - accuracy: 0.6429 - val_loss: 16.3333 - val_accuracy: 0.6500

Epoch 00002: val_accuracy did not improve from 0.66333
Epoch 3/50
22/22 [==============================] - 4s 171ms/step - loss: 10.7830 - accuracy: 0.6657 - val_loss: 15.0050 - val_accuracy: 0.6567

Epoch 00003: val_accuracy did not improve from 0.66333
Epoch 4/50
22/22 [==============================] - 4s 156ms/step - loss: 6.3144 - accuracy: 0.7329 - val_loss: 8.3161 - val_accuracy: 0.7133

Epoch 00004: val_accuracy improved from 0.66333 to 0.71333, saving model to hyperparameter_adam.h5
Epoch 5/50
22/22 [==============================] - 4s 149ms/step - loss: 4.8127 - accuracy: 0.7471 - val_loss: 6.215

In [38]:
adam_model.evaluate(test_dataset)

1/1 [==============================] - 7s 7s/step - loss: 2.2028 - accuracy: 0.7333


[2.2028489112854004, 0.7333333492279053]

# Transfer Learning and tune the hyperparameters with frozen layers (SGD optimizer)

In [23]:
def model_builder_SGD(hp):
    
    hp_units = hp.Choice('units', [16, 32, 64, 128, 256, 512, 1024])
    hp_lr = hp.Choice('lr', [1e-2, 1e-3, 1e-4])
    hp_ratio1 = hp.Choice('ratio1', [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8])
    hp_ratio2 = hp.Choice('ratio2', [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8])
    hp_activation = hp.Choice('activation', ['relu', 'elu'])
    
    inputs = tf.keras.Input(shape=(224, 224, 3))
    
    x = data_augmentation(inputs)
    x = preprocess_input(x)
    x = base_model(x, training=False)
    x = Flatten()(x)
    x = tf.keras.layers.Dropout(hp_ratio1)(x)
    x = tf.keras.layers.Dense(units=hp_units, activation=hp_activation)(x)
    x = tf.keras.layers.Dropout(hp_ratio2)(x)
    prediction = Dense(10, activation='softmax')(x)
    model = tf.keras.Model(inputs=inputs, outputs=prediction)
    
    model.compile(
        optimizer=tf.keras.optimizers.SGD(learning_rate=hp_lr),
        loss=tf.keras.losses.CategoricalCrossentropy(),
        metrics=['accuracy']
    )

    return model

In [33]:
tuner2 = kt.Hyperband(
    model_builder_SGD,
    objective='val_accuracy',
    max_epochs=10,
    directory="./",
    project_name='vgg_sgd'
)

stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)
tuner2.search(train_dataset, epochs=50,validation_data=validation_dataset, callbacks=[stop_early])

# Get the optimal hyperparameters
best_hps=tuner2.get_best_hyperparameters(num_trials=1)[0]

Trial 30 Complete [00h 00m 48s]
val_accuracy: 0.11999999731779099

Best val_accuracy So Far: 0.7233333587646484
Total elapsed time: 00h 10m 37s


In [34]:
print(best_hps.get('units'))
print(best_hps.get('lr'))
print(best_hps.get('ratio1'))
print(best_hps.get('ratio2'))
print(best_hps.get('activation'))

1024
0.01
0.5
0.1
relu


In [35]:
x = data_augmentation(inputs)
x = preprocess_input(x)
x = base_model(x, training=False)
x = Flatten()(x)
x = tf.keras.layers.Dropout(0.5)(x)
x = tf.keras.layers.Dense(units=1024, activation='relu')(x)
x = tf.keras.layers.Dropout(0.1)(x)
prediction = Dense(10, activation='softmax')(x)
model = tf.keras.Model(inputs=inputs, outputs=prediction)

model.compile(optimizer=tf.keras.optimizers.SGD(learning_rate= 0.01),
              loss=tf.keras.losses.CategoricalCrossentropy(),
              metrics=['accuracy'])

model_checkpoints = [
    tf.keras.callbacks.ModelCheckpoint(filepath = 'hyperparameter_sgd.h5', 
    verbose=1, save_best_only=True, save_weights_only=False, mode='max', monitor='val_accuracy') 
    ]

history = model.fit(train_dataset,epochs=50, validation_data = validation_dataset, callbacks = [model_checkpoints])


Epoch 1/50
22/22 [==============================] - 5s 161ms/step - loss: 39.2008 - accuracy: 0.3300 - val_loss: 1.7007 - val_accuracy: 0.5100

Epoch 00001: val_accuracy improved from -inf to 0.51000, saving model to hyperparameter_sgd.h5
Epoch 2/50
22/22 [==============================] - 4s 163ms/step - loss: 1.9000 - accuracy: 0.5414 - val_loss: 1.5716 - val_accuracy: 0.5567

Epoch 00002: val_accuracy improved from 0.51000 to 0.55667, saving model to hyperparameter_sgd.h5
Epoch 3/50
22/22 [==============================] - 4s 150ms/step - loss: 1.3649 - accuracy: 0.6229 - val_loss: 1.5206 - val_accuracy: 0.5867

Epoch 00003: val_accuracy improved from 0.55667 to 0.58667, saving model to hyperparameter_sgd.h5
Epoch 4/50
22/22 [==============================] - 4s 157ms/step - loss: 1.0865 - accuracy: 0.7000 - val_loss: 1.3758 - val_accuracy: 0.6600

Epoch 00004: val_accuracy improved from 0.58667 to 0.66000, saving model to hyperparameter_sgd.h5
Epoch 5/50
22/22 [====================

In [40]:
sgd_model.evaluate(test_dataset)

1/1 [==============================] - 3s 3s/step - loss: 1.5944 - accuracy: 0.6333


[1.5944373607635498, 0.6333333253860474]

# Transfer Learning and tune the hyperparameters with some unfrozen layers (Adam optimizer)

In [28]:
base_model.trainable = True
print("Number of layers in the base model: ", len(base_model.layers))

def model_builder_Adam_unfrozen(hp):
    
    hp_lr = hp.Choice('lr', [1e-2, 1e-3, 1e-4])
    hp_layers = hp.Choice('layers', [2,4,6,8,10,12,14,16,18,20,22])

    # Fine-tune from this layer onwards
    fine_tune_at = hp_layers

    # Freeze all the layers before the `fine_tune_at` layer
    for layer in base_model.layers[:fine_tune_at]:
      layer.trainable = False
    
    adam_model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate= hp_lr),
              loss=tf.keras.losses.CategoricalCrossentropy(),
              metrics=['accuracy'])

    return adam_model

Number of layers in the base model:  22


In [47]:
tuner4 = kt.Hyperband(
    model_builder_Adam_unfrozen,
    objective='val_accuracy',
    max_epochs=10,
    directory="./",
    project_name='vgg_adam_unfrozen'
)

stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)
tuner4.search(train_dataset, epochs=50,validation_data=validation_dataset, callbacks=[stop_early])

# Get the optimal hyperparameters
best_hps=tuner4.get_best_hyperparameters(num_trials=1)[0]

Trial 30 Complete [00h 00m 27s]
val_accuracy: 0.5400000214576721

Best val_accuracy So Far: 0.6100000143051147
Total elapsed time: 00h 12m 07s


In [48]:
print(best_hps.get('lr'))
print(best_hps.get('layers'))

0.001
14


In [50]:
base_model.trainable = True
fine_tune_at = 14

    # Freeze all the layers before the `fine_tune_at` layer
for layer in base_model.layers[:fine_tune_at]:
    layer.trainable = False
    
adam_model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate= 0.001),
              loss=tf.keras.losses.CategoricalCrossentropy(),
              metrics=['accuracy'])


fine_tune_epochs = 20
total_epochs =  50 + fine_tune_epochs


model_checkpoints = [
    tf.keras.callbacks.ModelCheckpoint(filepath = './hyperparameter_adam_unfrozen.h5', 
    verbose=1, save_best_only=True, save_weights_only=False, mode='max', monitor='val_accuracy') 
    ]

history_fine = adam_model.fit(train_dataset,
                         epochs=total_epochs,
                         initial_epoch=50,
                         validation_data=validation_dataset,
                             callbacks = [model_checkpoints])




Epoch 51/70
22/22 [==============================] - 5s 159ms/step - loss: 5.0329 - accuracy: 0.4429 - val_loss: 6.3353 - val_accuracy: 0.5300

Epoch 00051: val_accuracy improved from -inf to 0.53000, saving model to ./hyperparameter_adam_unfrozen.h5
Epoch 52/70
22/22 [==============================] - 4s 171ms/step - loss: 4.1229 - accuracy: 0.4629 - val_loss: 6.4080 - val_accuracy: 0.5233

Epoch 00052: val_accuracy did not improve from 0.53000
Epoch 53/70
22/22 [==============================] - 4s 152ms/step - loss: 4.4956 - accuracy: 0.4429 - val_loss: 6.4600 - val_accuracy: 0.5333

Epoch 00053: val_accuracy improved from 0.53000 to 0.53333, saving model to ./hyperparameter_adam_unfrozen.h5
Epoch 54/70
22/22 [==============================] - 4s 150ms/step - loss: 4.4011 - accuracy: 0.4471 - val_loss: 6.4091 - val_accuracy: 0.5067

Epoch 00054: val_accuracy did not improve from 0.53333
Epoch 55/70
22/22 [==============================] - 4s 150ms/step - loss: 3.6846 - accuracy: 0.4

In [52]:
adam_model_unfrozen = tf.keras.models.load_model("./hyperparameter_adam_unfrozen.h5")


adam_model_unfrozen.evaluate(test_dataset)


1/1 [==============================] - 4s 4s/step - loss: 13.3998 - accuracy: 0.4667


[13.399843215942383, 0.46666666865348816]

# Transfer Learning and tune the hyperparameters with some unfrozen layers (SGD optimizer)

In [53]:
base_model.trainable = True

In [54]:
print("Number of layers in the base model: ", len(base_model.layers))


Number of layers in the base model:  22


In [29]:
def model_builder_SGD_unfrozen(hp):
    
    hp_lr = hp.Choice('lr', [1e-2, 1e-3, 1e-4])
    hp_layers = hp.Choice('layers', [2,4,6,8,10,12,14,16,18,20,22])

    # Fine-tune from this layer onwards
    fine_tune_at = hp_layers

    # Freeze all the layers before the `fine_tune_at` layer
    for layer in base_model.layers[:fine_tune_at]:
      layer.trainable = False
    
    sgd_model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate= hp_lr),
              loss=tf.keras.losses.CategoricalCrossentropy(),
              metrics=['accuracy'])

    return sgd_model

In [59]:
tuner3 = kt.Hyperband(
    model_builder_SGD_unfrozen,
    objective='val_accuracy',
    max_epochs=10,
    directory="./",
    project_name='vgg_sgd_unfrozen'
)

stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)
tuner3.search(train_dataset, epochs=50,validation_data=validation_dataset, callbacks=[stop_early])

# Get the optimal hyperparameters
best_hps=tuner3.get_best_hyperparameters(num_trials=1)[0]

Trial 30 Complete [00h 00m 29s]
val_accuracy: 0.5766666531562805

Best val_accuracy So Far: 0.7599999904632568
Total elapsed time: 00h 05m 29s


In [60]:
print(best_hps.get('lr'))
print(best_hps.get('layers'))

0.0001
18


In [61]:
base_model.trainable = True
fine_tune_at = 18

    # Freeze all the layers before the `fine_tune_at` layer
for layer in base_model.layers[:fine_tune_at]:
    layer.trainable = False
    
sgd_model.compile(optimizer=tf.keras.optimizers.SGD(learning_rate= 0.0001),
              loss=tf.keras.losses.CategoricalCrossentropy(),
              metrics=['accuracy'])


fine_tune_epochs = 20
total_epochs =  50 + fine_tune_epochs


model_checkpoints = [
    tf.keras.callbacks.ModelCheckpoint(filepath = './hyperparameter_sgd_unfrozen.h5', 
    verbose=1, save_best_only=True, save_weights_only=False, mode='max', monitor='val_accuracy') 
    ]

history_fine = sgd_model.fit(train_dataset,
                         epochs=total_epochs,
                         initial_epoch=50,
                         validation_data=validation_dataset,
                             callbacks = [model_checkpoints])




Epoch 51/70
22/22 [==============================] - 5s 166ms/step - loss: 1.9145 - accuracy: 0.6900 - val_loss: 19.9133 - val_accuracy: 0.5533

Epoch 00051: val_accuracy improved from -inf to 0.55333, saving model to ./hyperparameter_sgd_unfrozen.h5
Epoch 52/70
22/22 [==============================] - 4s 147ms/step - loss: 3.2794 - accuracy: 0.6900 - val_loss: 19.7806 - val_accuracy: 0.5533

Epoch 00052: val_accuracy did not improve from 0.55333
Epoch 53/70
22/22 [==============================] - 4s 162ms/step - loss: 2.1960 - accuracy: 0.7014 - val_loss: 19.6774 - val_accuracy: 0.5533

Epoch 00053: val_accuracy did not improve from 0.55333
Epoch 54/70
22/22 [==============================] - 4s 149ms/step - loss: 3.3699 - accuracy: 0.6829 - val_loss: 19.5679 - val_accuracy: 0.5533

Epoch 00054: val_accuracy did not improve from 0.55333
Epoch 55/70
22/22 [==============================] - 4s 160ms/step - loss: 2.5240 - accuracy: 0.7171 - val_loss: 19.4484 - val_accuracy: 0.5533

Epoc

In [62]:
sgd_model_unfrozen = tf.keras.models.load_model("hyperparameter_sgd_unfrozen.h5")

In [63]:
sgd_model_unfrozen.evaluate(test_dataset)

1/1 [==============================] - 4s 4s/step - loss: 20.3343 - accuracy: 0.4667


[20.3343448638916, 0.46666666865348816]

# Add more training images by generating more images using GAN

In [9]:
materials = ['fabric','foliage','glass','leather','metal','paper','plastic','stone','water','wood']

path_to_folder = '../input/ganmaterials/cz4042_neural_networks_deep_learning-main/data/image'

train_dataset_GAN = tf.keras.utils.image_dataset_from_directory(
    directory=path_to_folder,
    labels='inferred',
    label_mode='categorical',
    class_names=materials,
    # color_mode='grayscale',
    batch_size=32,
    image_size=(224,224),
    shuffle=True,
    seed=42,
    validation_split=0.3,
    subset='training')
    
    
validation_dataset_GAN = tf.keras.utils.image_dataset_from_directory(
    directory=path_to_folder,
    labels='inferred',
    label_mode='categorical',
    class_names=materials,
    # color_mode='grayscale',
    batch_size=32,
    image_size=(224,224),
    shuffle=True,
    seed=42,
    validation_split=0.3,
    subset='validation')


Found 2000 files belonging to 10 classes.
Using 1400 files for training.
Found 2000 files belonging to 10 classes.
Using 600 files for validation.


## Transfer Learning and tune the hyperparameters with frozen layers with GAN (Adam optimizer)

In [18]:
tuner = kt.Hyperband(
    model_builder,
    objective='val_accuracy',
    max_epochs=10,
    directory="./",
    project_name = "adam_GAN"
)

stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)

tuner.search(train_dataset_GAN, epochs=50,validation_data=validation_dataset_GAN, callbacks=[stop_early])

# Get the optimal hyperparameters
best_hps=tuner.get_best_hyperparameters(num_trials=1)[0]

print(best_hps.get('units'))
print(best_hps.get('lr'))
print(best_hps.get('ratio1'))
print(best_hps.get('ratio2'))
print(best_hps.get('activation'))

Trial 30 Complete [00h 01m 19s]
val_accuracy: 0.1899999976158142

Best val_accuracy So Far: 0.7099999785423279
Total elapsed time: 00h 16m 58s
512
0.0001
0.1
0.7
elu


In [20]:
inputs = tf.keras.Input(shape=(224, 224, 3))
x = data_augmentation(inputs)
x = preprocess_input(x)
x = base_model(x, training=False)
x = Flatten()(x)
x = tf.keras.layers.Dropout(0.1)(x)
x = tf.keras.layers.Dense(units=512, activation='elu')(x)
x = tf.keras.layers.Dropout(0.7)(x)
prediction = Dense(10, activation='softmax')(x)
model = tf.keras.Model(inputs=inputs, outputs=prediction)

model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate= 0.0001),
              loss=tf.keras.losses.CategoricalCrossentropy(),
              metrics=['accuracy'])

model_checkpoints = [
    tf.keras.callbacks.ModelCheckpoint(filepath = 'hyperparameter_adam_GAN.h5', 
    verbose=1, save_best_only=True, save_weights_only=False, mode='max', monitor='val_accuracy') 
    ]

history = model.fit(train_dataset_GAN, epochs=50, validation_data = validation_dataset_GAN, callbacks = [model_checkpoints])


Epoch 1/50
44/44 [==============================] - 7s 135ms/step - loss: 17.2776 - accuracy: 0.2736 - val_loss: 5.1677 - val_accuracy: 0.5783

Epoch 00001: val_accuracy improved from -inf to 0.57833, saving model to hyperparameter_adam_GAN.h5
Epoch 2/50
44/44 [==============================] - 6s 131ms/step - loss: 6.1738 - accuracy: 0.4921 - val_loss: 2.6813 - val_accuracy: 0.6533

Epoch 00002: val_accuracy improved from 0.57833 to 0.65333, saving model to hyperparameter_adam_GAN.h5
Epoch 3/50
44/44 [==============================] - 7s 136ms/step - loss: 4.0245 - accuracy: 0.5421 - val_loss: 1.9307 - val_accuracy: 0.6617

Epoch 00003: val_accuracy improved from 0.65333 to 0.66167, saving model to hyperparameter_adam_GAN.h5
Epoch 4/50
44/44 [==============================] - 6s 133ms/step - loss: 3.5096 - accuracy: 0.5329 - val_loss: 1.7263 - val_accuracy: 0.6850

Epoch 00004: val_accuracy improved from 0.66167 to 0.68500, saving model to hyperparameter_adam_GAN.h5
Epoch 5/50
44/44 [

In [22]:
adam_model_GAN = tf.keras.models.load_model("./hyperparameter_adam_GAN.h5")
adam_model_GAN.evaluate(test_dataset)

1/1 [==============================] - 8s 8s/step - loss: 2.1804 - accuracy: 0.7000


[2.180387020111084, 0.699999988079071]

## Transfer Learning and tune the hyperparameters with frozen layers with GAN (SGD optimizer)

In [24]:
tuner2 = kt.Hyperband(
    model_builder_SGD,
    objective='val_accuracy',
    max_epochs=10,
    directory="./",
    project_name='sgd_GAN'
)

stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)
tuner2.search(train_dataset_GAN, epochs=50,validation_data=validation_dataset_GAN, callbacks=[stop_early])

# Get the optimal hyperparameters
best_hps=tuner2.get_best_hyperparameters(num_trials=1)[0]

print(best_hps.get('units'))
print(best_hps.get('lr'))
print(best_hps.get('ratio1'))
print(best_hps.get('ratio2'))
print(best_hps.get('activation'))

Trial 30 Complete [00h 01m 27s]
val_accuracy: 0.7683333158493042

Best val_accuracy So Far: 0.7799999713897705
Total elapsed time: 00h 18m 39s
1024
0.001
0.3
0.5
relu


In [25]:
inputs = tf.keras.Input(shape=(224, 224, 3))
x = data_augmentation(inputs)
x = preprocess_input(x)
x = base_model(x, training=False)
x = Flatten()(x)
x = tf.keras.layers.Dropout(0.3)(x)
x = tf.keras.layers.Dense(units=1024, activation='elu')(x)
x = tf.keras.layers.Dropout(0.5)(x)
prediction = Dense(10, activation='softmax')(x)
model = tf.keras.Model(inputs=inputs, outputs=prediction)

model.compile(optimizer=tf.keras.optimizers.SGD(learning_rate= 0.001),
              loss=tf.keras.losses.CategoricalCrossentropy(),
              metrics=['accuracy'])

model_checkpoints = [
    tf.keras.callbacks.ModelCheckpoint(filepath = 'hyperparameter_SGD_GAN.h5', 
    verbose=1, save_best_only=True, save_weights_only=False, mode='max', monitor='val_accuracy') 
    ]

history = model.fit(train_dataset_GAN, epochs=50, validation_data = validation_dataset_GAN, callbacks = [model_checkpoints])


Epoch 1/50
44/44 [==============================] - 7s 137ms/step - loss: 13.1082 - accuracy: 0.3793 - val_loss: 4.9996 - val_accuracy: 0.6433

Epoch 00001: val_accuracy improved from -inf to 0.64333, saving model to hyperparameter_SGD_GAN.h5
Epoch 2/50
44/44 [==============================] - 6s 130ms/step - loss: 6.3044 - accuracy: 0.5857 - val_loss: 3.8974 - val_accuracy: 0.6900

Epoch 00002: val_accuracy improved from 0.64333 to 0.69000, saving model to hyperparameter_SGD_GAN.h5
Epoch 3/50
44/44 [==============================] - 6s 132ms/step - loss: 4.4720 - accuracy: 0.6414 - val_loss: 3.1193 - val_accuracy: 0.7167

Epoch 00003: val_accuracy improved from 0.69000 to 0.71667, saving model to hyperparameter_SGD_GAN.h5
Epoch 4/50
44/44 [==============================] - 6s 132ms/step - loss: 3.5877 - accuracy: 0.6843 - val_loss: 2.4248 - val_accuracy: 0.7350

Epoch 00004: val_accuracy improved from 0.71667 to 0.73500, saving model to hyperparameter_SGD_GAN.h5
Epoch 5/50
44/44 [====

In [27]:
sgd_model_GAN = tf.keras.models.load_model("./hyperparameter_SGD_GAN.h5")
sgd_model_GAN.evaluate(test_dataset)

1/1 [==============================] - 4s 4s/step - loss: 1.4135 - accuracy: 0.7000


[1.4134563207626343, 0.699999988079071]

## Transfer Learning and tune the hyperparameters with some unfrozen layers with GAN (Adam optimizer)

In [31]:
base_model.trainable = True
print("Number of layers in the base model: ", len(base_model.layers))

def model_builder_Adam_unfrozen_GAN(hp):
    
    hp_lr = hp.Choice('lr', [1e-2, 1e-3, 1e-4])
    hp_layers = hp.Choice('layers', [2,4,6,8,10,12,14,16,18,20,22])

    # Fine-tune from this layer onwards
    fine_tune_at = hp_layers

    # Freeze all the layers before the `fine_tune_at` layer
    for layer in base_model.layers[:fine_tune_at]:
      layer.trainable = False
    
    adam_model_GAN.compile(optimizer=tf.keras.optimizers.Adam(learning_rate= hp_lr),
              loss=tf.keras.losses.CategoricalCrossentropy(),
              metrics=['accuracy'])

    return adam_model_GAN

tuner4 = kt.Hyperband(
    model_builder_Adam_unfrozen_GAN,
    objective='val_accuracy',
    max_epochs=10,
    directory="./",
    project_name='adam_unfrozen_GAN'
)

stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)
tuner4.search(train_dataset_GAN, epochs=50,validation_data=validation_dataset_GAN, callbacks=[stop_early])

# Get the optimal hyperparameters
best_hps=tuner4.get_best_hyperparameters(num_trials=1)[0]

Trial 30 Complete [00h 01m 28s]
val_accuracy: 0.2016666680574417

Best val_accuracy So Far: 0.5483333468437195
Total elapsed time: 00h 16m 36s


In [32]:
print(best_hps.get('layers'))
print(best_hps.get('lr'))

10
0.01


In [33]:
base_model.trainable = True
fine_tune_at = 10

    # Freeze all the layers before the `fine_tune_at` layer
for layer in base_model.layers[:fine_tune_at]:
    layer.trainable = False
    
adam_model_GAN.compile(optimizer=tf.keras.optimizers.Adam(learning_rate= 0.01),
              loss=tf.keras.losses.CategoricalCrossentropy(),
              metrics=['accuracy'])


fine_tune_epochs = 20
total_epochs =  50 + fine_tune_epochs


model_checkpoints = [
    tf.keras.callbacks.ModelCheckpoint(filepath = './hyperparameter_adam_unfrozen_GAN.h5', 
    verbose=1, save_best_only=True, save_weights_only=False, mode='max', monitor='val_accuracy') 
    ]

history_fine = adam_model_GAN.fit(train_dataset_GAN,
                         epochs=total_epochs,
                         initial_epoch=50,
                         validation_data=validation_dataset_GAN,
                             callbacks = [model_checkpoints])




Epoch 51/70
44/44 [==============================] - 7s 142ms/step - loss: 5.1829 - accuracy: 0.1750 - val_loss: 3.9045 - val_accuracy: 0.2317

Epoch 00051: val_accuracy improved from -inf to 0.23167, saving model to ./hyperparameter_adam_unfrozen_GAN.h5
Epoch 52/70
44/44 [==============================] - 6s 131ms/step - loss: 4.2695 - accuracy: 0.1507 - val_loss: 3.3291 - val_accuracy: 0.2100

Epoch 00052: val_accuracy did not improve from 0.23167
Epoch 53/70
44/44 [==============================] - 6s 131ms/step - loss: 4.8210 - accuracy: 0.1400 - val_loss: 2.2985 - val_accuracy: 0.2017

Epoch 00053: val_accuracy did not improve from 0.23167
Epoch 54/70
44/44 [==============================] - 6s 129ms/step - loss: 4.0054 - accuracy: 0.1714 - val_loss: 2.8654 - val_accuracy: 0.2117

Epoch 00054: val_accuracy did not improve from 0.23167
Epoch 55/70
44/44 [==============================] - 6s 130ms/step - loss: 4.1142 - accuracy: 0.1307 - val_loss: 2.8279 - val_accuracy: 0.1800

Epoc

In [34]:
adam_model_unfrozen_GAN = tf.keras.models.load_model("hyperparameter_adam_unfrozen_GAN.h5")

In [35]:
adam_model_unfrozen_GAN.evaluate(test_dataset)

1/1 [==============================] - 3s 3s/step - loss: 2.3755 - accuracy: 0.3000


[2.3754916191101074, 0.30000001192092896]

## Transfer Learning and tune the hyperparameters with some unfrozen layers with GAN (SGD optimizer)

In [36]:
def model_builder_SGD_unfrozen_GAN(hp):
    
    hp_lr = hp.Choice('lr', [1e-2, 1e-3, 1e-4])
    hp_layers = hp.Choice('layers', [2,4,6,8,10,12,14,16,18,20,22])

    # Fine-tune from this layer onwards
    fine_tune_at = hp_layers

    # Freeze all the layers before the `fine_tune_at` layer
    for layer in base_model.layers[:fine_tune_at]:
      layer.trainable = False
    
    sgd_model_GAN.compile(optimizer=tf.keras.optimizers.Adam(learning_rate= hp_lr),
              loss=tf.keras.losses.CategoricalCrossentropy(),
              metrics=['accuracy'])

    return sgd_model_GAN

In [40]:
tuner3 = kt.Hyperband(
    model_builder_SGD_unfrozen_GAN,
    objective='val_accuracy',
    max_epochs=10,
    directory="./",
    project_name='vgg_sgd_unfrozen_GAN'
)

stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)
tuner3.search(train_dataset_GAN, epochs=50,validation_data=validation_dataset_GAN, callbacks=[stop_early])

# Get the optimal hyperparameters
best_hps=tuner3.get_best_hyperparameters(num_trials=1)[0]

print(best_hps.get('layers'))
print(best_hps.get('lr'))

Trial 30 Complete [00h 01m 18s]
val_accuracy: 0.3466666638851166

Best val_accuracy So Far: 0.699999988079071
Total elapsed time: 00h 15m 18s
10
0.001


In [41]:
base_model.trainable = True
fine_tune_at = 10

    # Freeze all the layers before the `fine_tune_at` layer
for layer in base_model.layers[:fine_tune_at]:
    layer.trainable = False
    
sgd_model_GAN.compile(optimizer=tf.keras.optimizers.SGD(learning_rate= 0.001),
              loss=tf.keras.losses.CategoricalCrossentropy(),
              metrics=['accuracy'])


fine_tune_epochs = 20
total_epochs =  50 + fine_tune_epochs


model_checkpoints = [
    tf.keras.callbacks.ModelCheckpoint(filepath = './hyperparameter_sgd_unfrozen_GAN.h5', 
    verbose=1, save_best_only=True, save_weights_only=False, mode='max', monitor='val_accuracy') 
    ]

history_fine = sgd_model_GAN.fit(train_dataset_GAN,
                         epochs=total_epochs,
                         initial_epoch=50,
                         validation_data=validation_dataset_GAN,
                             callbacks = [model_checkpoints])




Epoch 51/70
44/44 [==============================] - 7s 135ms/step - loss: 3.2408 - accuracy: 0.2643 - val_loss: 2.1152 - val_accuracy: 0.3400

Epoch 00051: val_accuracy improved from -inf to 0.34000, saving model to ./hyperparameter_sgd_unfrozen_GAN.h5
Epoch 52/70
44/44 [==============================] - 7s 138ms/step - loss: 3.1409 - accuracy: 0.2786 - val_loss: 2.0712 - val_accuracy: 0.3400

Epoch 00052: val_accuracy did not improve from 0.34000
Epoch 53/70
44/44 [==============================] - 6s 132ms/step - loss: 2.9683 - accuracy: 0.2857 - val_loss: 2.0654 - val_accuracy: 0.3700

Epoch 00053: val_accuracy improved from 0.34000 to 0.37000, saving model to ./hyperparameter_sgd_unfrozen_GAN.h5
Epoch 54/70
44/44 [==============================] - 6s 129ms/step - loss: 2.8573 - accuracy: 0.2829 - val_loss: 2.0299 - val_accuracy: 0.3483

Epoch 00054: val_accuracy did not improve from 0.37000
Epoch 55/70
44/44 [==============================] - 6s 131ms/step - loss: 2.8026 - accurac

In [42]:
sgd_model_unfrozen_GAN = tf.keras.models.load_model("hyperparameter_sgd_unfrozen_GAN.h5")
sgd_model_unfrozen_GAN.evaluate(test_dataset)

1/1 [==============================] - 3s 3s/step - loss: 5.0617 - accuracy: 0.3000


[5.061696529388428, 0.30000001192092896]